Project ETL - NBA Data

In [1]:
import datetime
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from zodiac_sign import get_zodiac_sign

In [2]:
player_file = "Resources/player_data.csv"
player_data_df = pd.read_csv(player_file)
player_data_df.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [3]:
NBA_player_week_file = "Resources/NBA_player_of_the_week.csv"
NBA_player_week_df = pd.read_csv(NBA_player_week_file)
NBA_player_week_df.head()

,Active season,Player,Team,Conference,Date,Position,Height,Weight,Age,Draft Year,Seasons in league,Season,Season short,Real_value
0,0,Micheal Ray Richardson,New Jersey Nets,NaN,"Apr 14, 1985",PG,6-5,189,29,1978,6,1984-1985,1985,1.0
1,0,Derek Smith,Los Angeles Clippers,NaN,"Apr 7, 1985",SG,6-6,205,23,1982,2,1984-1985,1985,1.0
2,0,Calvin Natt,Denver Nuggets,NaN,"Apr 1, 1985",F,6-6,220,28,1979,5,1984-1985,1985,1.0
3,0,Kareem Abdul-Jabbar,Los Angeles Lakers,NaN,"Mar 24, 1985",C,7-2,225,37,1969,15,1984-1985,1985,1.0
4,0,Larry Bird,Boston Celtics,NaN,"Mar 17, 1985",SF,6-9,220,28,1978,5,1984-1985,1985,1.0


In [4]:
NBA_player_week_df.columns



# ### Transform player DataFrame

Index(['Active season', 'Player', 'Team', 'Conference', 'Date', 'Position',
       'Height', 'Weight', 'Age', 'Draft Year', 'Seasons in league', 'Season',
       'Season short', 'Real_value'],
      dtype='object')

In [5]:


new_player_data_df = player_data_df[['name', 'birth_date', 'height']].copy()
new_player_data_df.columns = ['player_name', 'birth_date', 'height']
new_player_data_df['id'] = new_player_data_df.index
new_player_data_df['name_height']= new_player_data_df['player_name']+'_'+new_player_data_df['height']

In [6]:


# Create a filtered dataframe from specific columns
df = new_player_data_df
new_NBA_player_week_df = NBA_player_week_df[["Player", "Age", "Height", "Real_value", "Team"]].copy()

new_NBA_player_week_df.columns = ["player_name", "age", "height", "real_value", "team"]

new_NBA_player_week_df.head()

,player_name,age,height,real_value,team
0,Micheal Ray Richardson,29,6-5,1.0,New Jersey Nets
1,Derek Smith,23,6-6,1.0,Los Angeles Clippers
2,Calvin Natt,28,6-6,1.0,Denver Nuggets
3,Kareem Abdul-Jabbar,37,7-2,1.0,Los Angeles Lakers
4,Larry Bird,28,6-9,1.0,Boston Celtics


In [7]:
df['birth_date'] = pd.to_datetime(df['birth_date'])
df = df.dropna()
df['zodiac'] = df['birth_date'].apply(get_zodiac_sign)
df.head()

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,player_name,birth_date,height,id,name_height,zodiac
0,Alaa Abdelnaby,1968-06-24,6-10,0,Alaa Abdelnaby_6-10,Cancer
1,Zaid Abdul-Aziz,1946-04-07,6-9,1,Zaid Abdul-Aziz_6-9,Aries
2,Kareem Abdul-Jabbar,1947-04-16,7-2,2,Kareem Abdul-Jabbar_7-2,Aries
3,Mahmoud Abdul-Rauf,1969-03-09,6-1,3,Mahmoud Abdul-Rauf_6-1,Pisces
4,Tariq Abdul-Wahad,1974-11-03,6-6,4,Tariq Abdul-Wahad_6-6,Scorpio


In [25]:
# ### Create database connection
rds_connection_string = "root:Musica123!@127.0.0.1/nba_players"
engine = create_engine(f'mysql://{rds_connection_string}')

In [26]:


# Confirm tables
engine.table_names()


# ### Load DataFrames into database

C:\Users\mbergamaschi\AppData\Local\Continuum\anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


['player_birth_date', 'player_week']

In [27]:


df.to_sql(name='player_birth_date', con=engine, if_exists='append', index=False)
new_NBA_player_week_df.to_sql(name='player_week', con=engine, if_exists='replace', index=False)

In [28]:

pd.read_sql_query('select * from player_week', con=engine).head()

,player_name,age,height,real_value,team
0,Micheal Ray Richardson,29,6-5,1.0,New Jersey Nets
1,Derek Smith,23,6-6,1.0,Los Angeles Clippers
2,Calvin Natt,28,6-6,1.0,Denver Nuggets
3,Kareem Abdul-Jabbar,37,7-2,1.0,Los Angeles Lakers
4,Larry Bird,28,6-9,1.0,Boston Celtics


In [29]:

pd.read_sql_query('select * from player_birth_date', con=engine).head()

,id,player_name,birth_date,height,name_height
0,0,Alaa Abdelnaby,1968-06-24,6-10,Alaa Abdelnaby_6-10
1,1,Zaid Abdul-Aziz,1946-04-07,6-9,Zaid Abdul-Aziz_6-9
2,2,Kareem Abdul-Jabbar,1947-04-16,7-2,Kareem Abdul-Jabbar_7-2
3,3,Mahmoud Abdul-Rauf,1969-03-09,6-1,Mahmoud Abdul-Rauf_6-1
4,4,Tariq Abdul-Wahad,1974-11-03,6-6,Tariq Abdul-Wahad_6-6
